In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

drw_crypto_market_prediction_path = kagglehub.competition_download('drw-crypto-market-prediction')

print('Data source import complete.')


<a id='Imports'></a>
# Imports

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from scipy.stats import pearsonr
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
train_df = pd.read_parquet("/kaggle/input/drw-crypto-market-prediction/train.parquet")
train_df.head(10)

,bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,...,X882,X883,X884,X885,X886,X887,X888,X889,X890,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-03-01 00:00:00,15.283,8.425,176.405,44.984,221.389,0.121263,-0.417690,0.005399,0.125948,0.058359,...,1.925423,1.847943,0.005676,0.190791,0.369691,0.377630,0.210153,0.159183,0.530636,0.562539
2023-03-01 00:01:00,38.590,2.336,525.846,321.950,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,...,1.928569,1.849468,0.005227,0.184660,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686
2023-03-01 00:02:00,0.442,60.250,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,...,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505
2023-03-01 00:03:00,4.865,21.016,335.742,124.963,460.705,0.072944,-0.436590,-0.102483,0.017551,0.007149,...,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703
2023-03-01 00:04:00,27.158,3.451,98.411,44.407,142.818,0.173820,-0.213489,0.096067,0.215709,0.107133,...,1.927084,1.848950,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452
2023-03-01 00:05:00,29.454,5.540,309.371,97.460,406.831,-0.315680,-1.084080,-0.854450,-0.750777,-0.379207,...,1.927302,1.849109,0.003692,0.162008,0.340401,0.362300,0.207264,0.158085,0.528800,0.100448
2023-03-01 00:06:00,3.478,29.514,100.608,233.525,334.133,-0.253388,-0.872270,-0.706323,-0.615929,-0.314296,...,1.927037,1.849043,0.003372,0.156788,0.334824,0.359307,0.206691,0.157866,0.528434,0.178679
2023-03-01 00:07:00,16.591,3.230,46.034,218.806,264.840,0.288073,0.191502,0.364450,0.459337,0.225287,...,1.926313,1.848763,0.003103,0.151744,0.329342,0.356341,0.206119,0.157648,0.528068,0.463684
2023-03-01 00:08:00,5.560,30.919,40.259,82.518,122.777,0.604698,0.749774,0.965517,1.074676,0.537434,...,1.924647,1.848043,0.002833,0.146853,0.323945,0.353397,0.205549,0.157430,0.527701,0.686066


In [ ]:
train_df.shape

(525887, 896)

In [ ]:
train_df.isnull().sum()

bid_qty     0
ask_qty     0
buy_qty     0
sell_qty    0
volume      0
           ..
X887        0
X888        0
X889        0
X890        0
label       0
Length: 896, dtype: int64

In [ ]:
test_df = pd.read_parquet("/kaggle/input/drw-crypto-market-prediction/test.parquet")

In [ ]:
test_df.shape

(538150, 896)

In [ ]:
train_df['imbalance']=(train_df['buy_qty']-train_df['sell_qty'])/(train_df['buy_qty']+train_df['sell_qty']+1e-6)

test_df['imbalance']=(test_df['buy_qty']-test_df['sell_qty'])/(test_df['buy_qty']+test_df['sell_qty']+1e-6)



In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split

from scipy.stats import pearsonr
import numpy as np

FEATURES = ["X863", "X856", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
            "X415", "X345", "X855", "X174", "X302", "X178", "X168", "X612",
            "X888", "X421", "X333","imbalance"]

LABEL_COLUMN = "label"

X=train_df[FEATURES]
y=train_df[LABEL_COLUMN]
X_test = test_df[FEATURES]

# Split data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "tree_method": "gpu_hist",
        "device": "cuda",
        "verbosity": 0,
        "random_state": 42,
        "n_jobs": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "max_leaves": trial.suggest_int("max_leaves", 8, 64),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 30),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.3, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 50.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 80.0),
        "n_estimators": 1667
    }

    model = xgb.XGBRegressor(early_stopping_rounds=50,**params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],


        verbose=False
    )

    preds = model.predict(X_valid)
    r, _ = pearsonr(y_valid, preds)

    # Maximize correlation
    return r

# Create and run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

#  results
print("Best Pearson correlation:", study.best_value)
print("Best Parameters:", study.best_params)


[I 2025-07-05 17:37:26,358] A new study created in memory with name: no-name-9e8e06e2-d90b-4a08-ac93-cfa86c1bd3a5
[I 2025-07-05 17:37:37,729] Trial 0 finished with value: 0.9015988698835706 and parameters: {'learning_rate': 0.11450078728256348, 'max_depth': 17, 'max_leaves': 64, 'min_child_weight': 8, 'subsample': 0.9779568217113908, 'colsample_bytree': 0.5012026561235785, 'colsample_bylevel': 0.46488381941255663, 'colsample_bynode': 0.4690483189685495, 'gamma': 0.8673086856090628, 'reg_alpha': 17.17141317052307, 'reg_lambda': 20.012773086142932}. Best is trial 0 with value: 0.9015988698835706.
[I 2025-07-05 17:37:46,070] Trial 1 finished with value: 0.8590704097383723 and parameters: {'learning_rate': 0.06793572238566362, 'max_depth': 16, 'max_leaves': 49, 'min_child_weight': 12, 'subsample': 0.8986073269245325, 'colsample_bytree': 0.8089918287477647, 'colsample_bylevel': 0.38341004855474237, 'colsample_bynode': 0.9844820159490963, 'gamma': 3.950688883131904, 'reg_alpha': 8.6762440127

Best Pearson correlation: 0.9638404214746017
Best Parameters: {'learning_rate': 0.2138910814136458, 'max_depth': 6, 'max_leaves': 42, 'min_child_weight': 27, 'subsample': 0.9465289683944015, 'colsample_bytree': 0.8710090055413087, 'colsample_bylevel': 0.7793963103875241, 'colsample_bynode': 0.8382256074060366, 'gamma': 0.02743232297636123, 'reg_alpha': 10.640311986292705, 'reg_lambda': 4.167358182384419}


In [ ]:
best_params = study.best_params

# Add required non-tuned params
best_params.update({
    "tree_method": "hist",
    "device": "cuda",
    "random_state": 42,
    "n_jobs": -1,
    "early_stopping_rounds": 50,
    "eval_metric": "rmse"
})

final_model = xgb.XGBRegressor(**best_params)
final_model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    verbose=False
)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=0.7793963103875241,
             colsample_bynode=0.8382256074060366,
             colsample_bytree=0.8710090055413087, device='cuda',
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0.02743232297636123,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2138910814136458,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=42,
             min_child_weight=27, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
#XGboost preds
preds_X_valid = final_model.predict(X_valid)
preds_X_train = final_model.predict(X_train)
preds_X_test = final_model.predict(X_test)

In [ ]:
#feature augmentation

X_train_combined=X_train.copy()
X_train_combined["xg_pred"]=preds_X_train

X_valid_combined=X_valid.copy()
X_valid_combined["xg_pred"]=preds_X_valid

X_test_combined=X_test.copy()
X_test_combined["xg_pred"]=preds_X_test

In [ ]:
# fit SGD
sgd = SGDRegressor(max_iter=500, tol = 1e-3, random_state=42)
sgd.fit(X_train_combined, y_train)

SGDRegressor(max_iter=500, random_state=42)

In [ ]:
#final preds
preds = sgd.predict(X_test_combined)
print(len(preds))

538150


In [ ]:
# printtttt !!!!
print(preds)

[-0.57604167  0.92263589  0.23959578 ...  0.16210565 -1.11081578
  0.31898413]


In [ ]:
# more checking :0 for improvements ( meh )
# Out-of-fold XGBoost predictions
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train))

for train_idx, valid_idx in kf.split(X_train):
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train.iloc[train_idx], y_train.iloc[train_idx],
              eval_set=[(X_train.iloc[valid_idx], y_train.iloc[valid_idx])],
              verbose=False)
    oof_preds[valid_idx] = model.predict(X_train.iloc[valid_idx])

X_train_combined = X_train.copy()
X_train_combined["xg_pred"] = oof_preds

# Fit final XGBoost on all training data for test predictions
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
X_valid_combined = X_valid.copy()
X_valid_combined["xg_pred"] = final_model.predict(X_valid)
X_test_combined = X_test.copy()
X_test_combined["xg_pred"] = final_model.predict(X_test)

# Scale features for SGD
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_valid_scaled = scaler.transform(X_valid_combined)
X_test_scaled = scaler.transform(X_test_combined)

# SGD model
sgd = SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
sgd.fit(X_train_scaled, y_train)
preds = sgd.predict(X_test_scaled)

# Submission
submission = pd.read_csv("/kaggle/input/drw-crypto-market-prediction/sample_submission.csv")
submission["prediction"] = preds
submission.to_csv("submission.csv", index=False)